# Demonstrating the reusability of [importnb](https://github.com/deathbeds/importnb).

* `importnb` imports notebooks as modules
* `importnb` allows notebooks to be discovered as tests.

    >>> with Notebook(): import test_notebook as nb
    >>> assert 'test_function' in dir(nb)

In [6]:
    from importnb import Notebook

In [7]:
    __all__ = 'test_function',

## pytest will discover `test_function` because it starts with `test_*`

In [12]:
    def test_function():
        assert True

## doctest can test the current notebook state.

In [10]:
    if __name__ == '__main__':
        import doctest
        doctest.testmod()

## doctest can test the future state of the module.

In [11]:
    if __name__ == '__main__':
        with Notebook():
            import test_notebook as nb
            doctest.testmod(nb)